In [84]:
import os
import kagglehub
import pandas as pd
import joblib

from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from imblearn.over_sampling import SMOTE

In [85]:
import mlflow
mlflow.set_tracking_uri("http://localhost:8080")


In [86]:

# Save dir
import os
SAVEDIR = os.getenv('ARTIFACT_DIR', '.') + '/saved_models'
os.makedirs(SAVEDIR, exist_ok=True)

def load_data():
    path = kagglehub.dataset_download("itsmesunil/bank-loan-modelling")
    df = pd.read_excel(
        os.path.join(path, "Bank_Personal_Loan_Modelling.xlsx"),
        sheet_name='Data'
    )
    # DROP via keyword axis=
    return df.drop(['ID', 'ZIP Code'], axis=1)


In [87]:

def split_data(df):
    X = df.drop('Personal Loan', axis=1)
    y = df['Personal Loan']
    X0, X_test, y0, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X0, y0, test_size=0.2, stratify=y0, random_state=42
    )
    return X_train, X_val, X_test, y_train, y_val, y_test


In [88]:

def preprocess_fit(X_train, X_val, X_test):
    rb = ['CCAvg','Mortgage']
    st = ['Income','Experience','Age']
    pt = PowerTransformer('yeo-johnson')
    rs = RobustScaler()
    ss = StandardScaler()

    # fit & transform train
    X_train[rb] = rs.fit_transform(pt.fit_transform(X_train[rb]))
    X_train[st] = ss.fit_transform(X_train[st])

    # transform val/test
    for X in (X_val, X_test):
        X[rb] = rs.transform(pt.transform(X[rb]))
        X[st] = ss.transform(X[st])

    # save transformers
    joblib.dump(pt, os.path.join(SAVEDIR, 'pt.pkl'))
    joblib.dump(rs, os.path.join(SAVEDIR, 'rs.pkl'))
    joblib.dump(ss, os.path.join(SAVEDIR, 'ss.pkl'))

    return X_train, X_val, X_test


In [89]:

def feature_select_fit(X_train, y_train, X_val, X_test):
    selector = RFE(LogisticRegression(max_iter=1000), n_features_to_select=8)
    Xtr = selector.fit_transform(X_train, y_train)
    Xv  = selector.transform(X_val)
    Xt  = selector.transform(X_test)
    joblib.dump(selector, os.path.join(SAVEDIR, 'selector.pkl'))
    return Xtr, Xv, Xt


In [90]:

def balance(X, y):
    sm = SMOTE(random_state=42)
    return sm.fit_resample(X, y)


In [91]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from mlflow.tracking import MlflowClient

def tune_and_save(X, y, X_val, y_val):
    mlflow.set_experiment("Minfy_ML_FLOW")

    best_f1 = 0
    best_model_name = None
    best_model_path = None

    grids = {
        'LogisticRegression': {
            'model': LogisticRegression(max_iter=1000),
            'params': {'C':[0.01,0.1,1,10], 'penalty':['l1','l2'], 'solver':['liblinear']}
        },
        'DecisionTree': {
            'model': DecisionTreeClassifier(),
            'params': {'max_depth':[3,5,7,None], 'min_samples_split':[2,5,10], 'min_samples_leaf':[1,2,4]}
        },
        'RandomForest': {
            'model': RandomForestClassifier(random_state=42),
            'params': {'n_estimators':[50,100], 'max_depth':[5,10,None]}
        },
        'GradientBoosting': {
            'model': GradientBoostingClassifier(random_state=42),
            'params': {'n_estimators':[50,100], 'learning_rate':[0.01,0.1]}
        },
        'KNN': {
            'model': KNeighborsClassifier(),
            'params': {'n_neighbors':[3,5,7]}
        },
        'SVM': {
            'model': SVC(probability=True, random_state=42),
            'params': {'C':[0.1,1,10], 'kernel':['linear','rbf']}
        }
    }

    for name, cfg in grids.items():
        with mlflow.start_run(run_name=name, nested=True) as run:
            mlflow.sklearn.autolog()

            gs = GridSearchCV(cfg['model'], cfg['params'], scoring='f1', cv=5, n_jobs=-1)
            gs.fit(X, y)
            best_model = gs.best_estimator_

            # Predict on validation set
            preds = best_model.predict(X_val)
            acc = accuracy_score(y_val, preds)
            prec = precision_score(y_val, preds)
            rec = recall_score(y_val, preds)
            f1 = f1_score(y_val, preds)

            mlflow.log_metrics({
                "val_accuracy": acc,
                "val_precision": prec,
                "val_recall": rec,
                "val_f1": f1
            })

            print(f"{name} tuned → {gs.best_params_}")
            print(f"→ Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")

            # Save locally and log model
            model_path = os.path.join(SAVEDIR, f"{name}_model.pkl")
            joblib.dump(best_model, model_path)

            mlflow.sklearn.log_model(best_model, artifact_path="model")

            # Track best model
            if f1 > best_f1:
                best_f1 = f1
                best_model_name = name
                best_model_path = f"runs:/{run.info.run_id}/model"

    # Register the best model in registry
    if best_model_name and best_model_path:
        print(f"\n🏆 Registering best model: {best_model_name} with F1 = {best_f1:.3f}")
        result = mlflow.register_model(
            model_uri=best_model_path,
            name="BankLoanBestModel"
        )
        print(f"✅ Registered model: {result.name}, version: {result.version}")


In [92]:
def main():
    mlflow.set_experiment("Minfy_ML_FLOW")

    with mlflow.start_run(run_name="Preprocessing and Tuning"):
        df = load_data()
        Xtr, Xv, Xt, ytr, yv, yt = split_data(df)
        Xtr, Xv, Xt = preprocess_fit(Xtr, Xv, Xt)
        Xtf, Xvf, Xsf = feature_select_fit(Xtr, ytr, Xv, Xt)

        # Log preprocessing artifacts
        for file in ['pt.pkl', 'rs.pkl', 'ss.pkl', 'selector.pkl']:
            mlflow.log_artifact(os.path.join(SAVEDIR, file))

        Xb, yb = balance(Xtf, ytr)
        tune_and_save(Xb, yb, Xvf, yv)


In [93]:
if __name__=='__main__':
    main()

2025/07/01 16:36:09 INFO mlflow.tracking.fluent: Experiment with name 'Minfy_ML_FLOW' does not exist. Creating a new experiment.
2025/07/01 16:36:28 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2025/07/01 16:36:29 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/07/01 16:36:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/01 16:36:32 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2025/07/01 16:36:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' obje

LogisticRegression tuned → {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
→ Accuracy: 0.884, Precision: 0.446, Recall: 0.866, F1: 0.589


2025/07/01 16:36:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/01 16:36:45 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


🏃 View run LogisticRegression at: http://localhost:8080/#/experiments/254295459546593827/runs/886c253011e54386acdcf7f275b08bf5
🧪 View experiment at: http://localhost:8080/#/experiments/254295459546593827


2025/07/01 16:36:52 INFO mlflow.sklearn.utils: Logging the 5 best runs, 31 runs will be omitted.
2025/07/01 16:36:52 WARNING mlflow.sklearn: Encountered exception during creation of child runs for parameter search. Child runs may be missing. Exception: 'NoneType' object has no attribute '_to_mlflow_entity'
2025/07/01 16:36:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

SVM tuned → {'C': 10, 'kernel': 'rbf'}
→ Accuracy: 0.977, Precision: 0.918, Recall: 0.836, F1: 0.875


2025/07/01 16:37:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SVM at: http://localhost:8080/#/experiments/254295459546593827/runs/ceedd82863f844c09ebb3166001967a9
🧪 View experiment at: http://localhost:8080/#/experiments/254295459546593827

🏆 Registering best model: RandomForest with F1 = 0.903


Successfully registered model 'BankLoanBestModel'.
2025/07/01 16:37:50 WARNING mlflow.tracking._model_registry.fluent: Run with id b9f9bd30d63745ac82d4e4f89f25891c has no artifacts at artifact path 'model', registering model based on models:/m-a32249315d874ac2946a36c76be0dd07 instead
2025/07/01 16:37:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BankLoanBestModel, version 1
Created version '1' of model 'BankLoanBestModel'.


✅ Registered model: BankLoanBestModel, version: 1
🏃 View run Preprocessing and Tuning at: http://localhost:8080/#/experiments/254295459546593827/runs/77de59b6050744c1b03e91fbb01adf7e
🧪 View experiment at: http://localhost:8080/#/experiments/254295459546593827
